In [1]:
import random
import torch

import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
import numpy as np
from warpctc_pytorch import CTCLoss
import os
import utils
import dataset
import crnn

In [2]:
dset = dataset.Dataset(img_root='/home/sherlock/Documents/express_recognition/data/train/telephone/', 
                       txt_root='/home/sherlock/Documents/express_recognition/data/train/telephone_label_train.txt')
dataloader = DataLoader(dset, batch_size=32, num_workers=4, collate_fn=dataset.alignCollate())

In [5]:
nh = 100
alphabet = '0123456789'
nclass = len(alphabet) + 1
nc = 1

converter = utils.strLabelConverter(alphabet)
criterion = CTCLoss()

mynet = crnn.CRNN(32, nc, nclass, nh)

mynet.cuda()

criterion = criterion.cuda()

loss_avg = utils.averager()
optimizer = optim.Adadelta(crnn.parameters(), lr=1e-3)

def trainBatch(crnn, criterion, optimizer):
    data = train_iter.next()
    cpu_images, cpu_texts = data
    batch_size = cpu_images.size(0)
    image = torch.FloatTensor(cpu_images)
    image = Variable(image).cuda()
    t, le = converter.encode(cpu_texts)
    text = torch.IntTensor(t)
    text = Variable(text)
    length = torch.IntTensor(le)
    length = Variable(length)
    preds = crnn(image)
    preds_size = Variable(torch.IntTensor([preds.size(0)] * batch_size))
    cost = criterion(preds, text, preds_size, length) / batch_size
    crnn.zero_grad()
    cost.backward()
    optimizer.step()
    return cost


for epoch in range(10000):
    train_iter = iter(dataloader)
    i = 0
    while i < len(dataloader):
        for p in mynet.parameters():
            p.requires_grad = True
        mynet.train()

        cost = trainBatch(mynet, criterion, optimizer)
        loss_avg.add(cost)
        i += 1

        if i % 10 == 0:
            print('[%d/%d][%d/%d] Loss: %f' %
                  (epoch, 10, i, len(dataloader), loss_avg.val()))
            loss_avg.reset()

#         if i % opt.valInterval == 0:
#             val(crnn, test_dataset, criterion)

        # do checkpointing
#         if i % opt.saveInterval == 0:
#             torch.save(
#                 crnn.state_dict(),
#                 '{0}/netCRNN_{1}_{2}.pth'.format(opt.experiment, epoch, i))


AttributeError: 'CRNN' object has no attribute 'CRNN'

In [3]:
nh = 100
alphabet = '0123456789'
nclass = len(alphabet) + 1
nc = 1
batch_size = 1

converter = utils.strLabelConverter(alphabet)
criterion = CTCLoss()

mynet = crnn.CRNN(32, nc, nclass, nh)

mynet.cuda()

loss_avg = utils.averager()
optimizer = optim.Adadelta(mynet.parameters(), lr=1e-3)

In [4]:
for epoch in range(10000):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 1):
        img, label = data
        img = Variable(img).cuda()
        t, le = converter.encode(label)
        text = torch.IntTensor(t)
        text = Variable(text)
        length = torch.IntTensor(le)
        length = Variable(length)
        preds = mynet(img)
        preds_size = Variable(torch.IntTensor([preds.size(0)] * batch_size))
        loss = criterion(preds, text, preds_size, length)
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss
        if i % 100 == 0:
            print('{}, Loss: {}'.format(epoch, running_loss / (i * batch_size)))
    print()
print('Finish Training!')

RuntimeError: cuda runtime error (2) : out of memory at /home/sherlock/Documents/pytorch/torch/lib/THC/generic/THCTensorMath.cu:35

In [8]:
def trainBatch(net, criterion, optimizer):
    data = train_iter.next()
    cpu_images, cpu_texts = data
    batch_size = cpu_images.size(0)
    image = torch.FloatTensor(cpu_images)
    image = Variable(image).cuda()
    t, le = converter.encode(cpu_texts)
    text = torch.IntTensor(t)
    text = Variable(text)
    length = torch.IntTensor(le)
    length = Variable(length)
    preds = net(image)
    preds_size = Variable(torch.IntTensor([preds.size(0)] * batch_size))
    cost = criterion(preds, text, preds_size, length) / batch_size
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    return cost


for epoch in range(10000):
    train_iter = iter(dataloader)
    i = 0
    while i < len(dataloader):
        mynet.train()
        try:
            cost = trainBatch(mynet, criterion, optimizer)
            loss_avg.add(cost)
            i += 1
        except:
            pass
        if i % 10 == 0:
            print('[%d/%d][%d/%d] Loss: %f' %
                  (epoch, 10, i, len(dataloader), loss_avg.val()))
            loss_avg.reset()

#         if i % opt.valInterval == 0:
#             val(crnn, test_dataset, criterion)

        # do checkpointing
#         if i % opt.saveInterval == 0:
#             torch.save(
#                 crnn.state_dict(),
#                 '{0}/netCRNN_{1}_{2}.pth'.format(opt.experiment, epoch, i))


[0/10][10/52] Loss: 51.295145
[0/10][20/52] Loss: 50.981127
[0/10][30/52] Loss: 50.891875
[0/10][40/52] Loss: 50.765867
[0/10][50/52] Loss: 50.616561


KeyboardInterrupt: 

In [ ]:
import PIL.Image as Image
img_path = '../data/train/telephone/telephone/18.png'
transformer = dataset.resizeNormalize((128, 32))
image = Image.open(img_path).convert('L')
image = transformer(image).cuda()
image = image.view(1, *image.size())
image = Variable(image)

mynet.eval()
preds = mynet(image)
_, preds = preds.max(2)
preds = preds.squeeze(2)
preds = preds.transpose(1, 0).contiguous().view(-1)
preds_size = Variable(torch.IntTensor([preds.size(0)]))
raw_pred = converter.decode(preds.data, preds_size.data, raw=True)
sim_pred = converter.decode(preds.data, preds_size.data, raw=False)
print('%-20s => %-20s' % (raw_pred, sim_pred))